# Eat Safe, Love

## Notebook Set Up

In [9]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [10]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [11]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [12]:
# review the collections in our database
collections = db.list_collection_names()

In [13]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [14]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {hygiene_20_count}")
# Display the first document in the results using pprint
first_hygiene_20 = establishments.find_one(query)
pprint(first_hygiene_20)

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67841617af8a9685c33c33f0'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 

In [15]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  67841617af8a9685c33c33f0   110681                  0   
1  67841618af8a9685c33c3766   612039                  0   
2  67841618af8a9685c33c3a70   730933                  0   
3  67841618af8a9685c33c3c61   172735                  0   
4  67841618af8a9685c33c3c6d   172953                  0   
5  67841618af8a9685c33c460f   512854                  0   
6  67841618af8a9685c33c4830  1537089                  0   
7  67841618af8a9685c33c5d59   155648                  0   
8  67841618af8a9685c33c619c  1012883                  0   
9  67841618af8a9685c33c69b2   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [16]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Match any Local Authority name containing 'London'
    "RatingValue": {"$gte": 4}
}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue >= 4:", count)
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with RatingValue >= 4: 0
None


In [17]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [18]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
penang_query = {"BusinessName": "Penang Flavours"}
penang_location = establishments.find_one(penang_query)
latitude = penang_location['geocode']['latitude']
longitude = penang_location['geocode']['longitude']

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
for result in results:
    pprint(result)

In [19]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)
df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [20]:
# Create a pipeline that:
pipeline = [
# 1. Matches establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
# 2. Groups the matches by Local Authority
     {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
# 3. Sorts the matches from highest to lowest
 {"$sort": {"count": -1}}
]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of Local Authorities with hygiene score of 0:", len(results))
# Print the first 10 results
print("Top 10 Local Authorities with highest number of hygiene score 0 establishments:")
for result in results[:10]:
    print(result)

Number of Local Authorities with hygiene score of 0: 56
Top 10 Local Authorities with highest number of hygiene score 0 establishments:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [21]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 56
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
